In [2]:
import requests as req
from bs4 import BeautifulSoup as bs
import pandas as pd
import re
import numpy as np
import datetime as dt
from datetime import datetime
import os

from time import sleep
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [10]:
def GetDistrictUrl(city, district):
    url = "https://en.aruodas.lt/"
    driver = webdriver.Chrome(r"../chromedriver.exe")
    driver.get(url)
    sleep(1)
    driver.find_element(By.ID, 'onetrust-accept-btn-handler').click()
    sleep(1)   
    driver.find_element(By.ID, 'display_FRegion').click()
    sleep(0.5)
    cityInput = driver.find_element(By.ID, "filterInput_FRegion")
    cityInput.send_keys(city)
    sleep(0.5)
    cityInput.send_keys(Keys.ENTER)
    sleep(0.5)
    driver.find_element(By.ID, 'display_FQuartal').click()
    sleep(2)
    driver.switch_to.frame(driver.find_element(By.ID, "fancybox-frame"))
    sleep(1)
    driver.find_element(By.XPATH,"//label[@title='{}']".format(district)).click()
    sleep(0.5)
    driver.find_element(By.ID, 'btSaveSelected').click()
    sleep(0.5)
    driver.switch_to.default_content()
    sleep(0.5)
    driver.find_element(By.ID, 'buttonSearchForm').click()
    currentUrl = driver.current_url
    driver.close()
    return currentUrl

In [11]:
def GetDistrictPropertyData(districtUrl, city, district, globalDataFrame):
    driver = webdriver.Chrome(r"../chromedriver.exe")
    driver.get(districtUrl)
    sleep(1)
    driver.find_element(By.ID,'onetrust-accept-btn-handler').click()
    sleep(1)
    
    soup = bs(driver.page_source, "lxml")
    pagination = soup.find("div", {"class" : "pagination"})
    maxPages = 1
    if(pagination is not None and len(pagination) > 0):
        maxPages = (len(pagination.find_all("a", {"class" : "page-bt"})))
    
    for page in range(1,maxPages+1):
        url = districtUrl + "puslapis/" + str(page) + "/"       
        driver.get(url)
        sleep(1)
        soup = bs(driver.page_source, "lxml")
        allElements = soup.find_all("tr", {"class" : "list-row"})
        allListings = [element.find('h3').find('a', href=True)['href'] for element in allElements if element.find("td", {"class":"list-adress"}) != None]
        
        for propertyUrl in allListings:
            driver.get(propertyUrl)
            sleep(1)
            soup = bs(driver.page_source, "lxml")
            
            price = soup.find("span", {"class" : "price-eur"}).text.split("€")[0].strip().replace(" ", "")
            pricePerM2 = soup.find("span", {"class" : "price-per"}).text.split("€")[0].strip().split("(")[1].replace(" ", "")         
            
            mainTable = soup.find("dl", {"class" : "obj-details"})
            area = mainTable.find('dt', text=re.compile('Area')).find_next_sibling('dd').text.strip().split(" ")[0].replace(",", ".")    
            floor = mainTable.find('dt', text=re.compile('Floor')).find_next_sibling('dd').text.strip()
            numberOfFloors = mainTable.find('dt', text=re.compile('No. of floors')).find_next_sibling('dd').text.strip()
            roomCount = mainTable.find('dt', text=re.compile('Number of rooms')).find_next_sibling('dd').text.strip()
            builtYear = mainTable.find('dt', text=re.compile('Build year')).find_next_sibling('dd').text.strip()
            heating = mainTable.find('dt', text=re.compile('Heating system')).find_next_sibling('dd').text.strip()
            equipment = mainTable.find('dt', text=re.compile('Equipment')).find_next_sibling('dd').text.strip()
            dataTable = pd.DataFrame({'Price': [price], 'PricePerM2': [pricePerM2], 'Area': [area], 'floor' : [floor], 'Number of floors': [numberOfFloors], 'Room count': [roomCount], 
                                      'Built year': [builtYear], 'Heating': [heating], 'Equipment': [equipment], 'City': [city], 'District': [district], 'href':[propertyUrl]})
            globalDataFrame = pd.concat( [globalDataFrame, dataTable])
            #print(globalDataFrame)
            driver.back()
            sleep(1)        
    driver.close()
    return globalDataFrame

In [295]:
def ConvertToKm(measurement):
    if(re.search('km', measurement) == None): 
        return (float)(measurement[:-1]) * 0.001
    else:
        return (float)(measurement[:-2])

In [307]:
def GetOneSpecificDistrictPropertyData(href, city, district, globalDataFrame):
    driver = webdriver.Chrome(r"../chromedriver.exe")
    driver.get(href)
    sleep(1)
    driver.find_element(By.ID,'onetrust-accept-btn-handler').click()
    sleep(1)
    soup = bs(driver.page_source, "lxml")
    price = soup.find("span", {"class" : "price-eur"}).text.split("€")[0].strip().replace(" ", "")
    pricePerM2 = soup.find("span", {"class" : "price-per"}).text.split("€")[0].strip().split("(")[1].replace(" ", "")         

    mainTable = soup.find("dl", {"class" : "obj-details"})
    area = mainTable.find('dt', text=re.compile('Area')).find_next_sibling('dd').text.strip().split(" ")[0].replace(",", ".")    
    floor = mainTable.find('dt', text=re.compile('Floor')).find_next_sibling('dd').text.strip()
    numberOfFloors = mainTable.find('dt', text=re.compile('No. of floors')).find_next_sibling('dd').text.strip()
    roomCount = mainTable.find('dt', text=re.compile('Number of rooms')).find_next_sibling('dd').text.strip()
    builtYear = mainTable.find('dt', text=re.compile('Build year')).find_next_sibling('dd').text.strip()
    heating = mainTable.find('dt', text=re.compile('Heating system')).find_next_sibling('dd').text.strip()
    equipment = mainTable.find('dt', text=re.compile('Equipment')).find_next_sibling('dd').text.strip()
    equipment = mainTable.find('dt', text=re.compile('Equipment')).find_next_sibling('dd').text.strip()
    #kindergarten = soup.find("span", {"class" : "cell-text"})
    nearestObj = soup.find_all("div", {"class" : "statistic-info-cell-main"})
    nearestKindergarten = [item.find("span",  {"class" : "cell-data"}).text for item in nearestObj if item.find("span", {"class" : "cell-text"}).text == 'Nearest kindergarten '][0].replace(' ', '')
    nearestKindergarten = nearestKindergarten.replace('\n', '')[1:].replace(",", ".")
    nearestEducational = [item.find("span",  {"class" : "cell-data"}).text for item in nearestObj if item.find("span", {"class" : "cell-text"}).text == 'Nearest educational institution'][0].replace(' ', '')
    nearestEducational = nearestEducational.replace('\n', '')[1:].replace(",", ".")
    nearestShop = [item.find("span",  {"class" : "cell-data"}).text for item in nearestObj if item.find("span", {"class" : "cell-text"}).text == 'Nearest shop'][0].replace(' ', '')
    nearestShop = nearestShop.replace(' ', '').replace('\n', '')[1:].replace(",", ".")
    nearestStop = [item.find("span",  {"class" : "cell-data"}).text for item in nearestObj if item.find("span", {"class" : "cell-text"}).text == 'Public transport stop'][0].replace(' ', '')
    nearestStop = nearestStop.replace('\n', '')[1:].replace(",", ".")
    crimeRate = soup.find("div", {"class" : "arrow_line_left"}).find("span", {"class" : "cell-data"}).text
    
    dataTable = pd.DataFrame({'Price': [price], 'PricePerM2': [pricePerM2], 'Area': [area], 'floor' : [floor], 'Number of floors': [numberOfFloors], 'Room count': [roomCount]
                              ,'Built year': [builtYear], 'Heating': [heating], 'Equipment': [equipment],'NearestKindergarten': [ConvertToKm(nearestKindergarten)]
                              ,'NearestEducational': [ConvertToKm(nearestEducational)],'NearestShop': [ConvertToKm(nearestShop)],'NearestStop': [ConvertToKm(nearestStop)]
                              ,'CrimeRate': [crimeRate],'City': [city], 'District': [district], 'href':[href]})
    
    globalDataFrame = pd.concat( [globalDataFrame, dataTable])
    sleep(1)
    driver.close()
    return globalDataFrame

In [308]:
def GetAllListings(flatsPriceData):
    global globalDataFrame
    for index in range(0,1):
        districtUrl = GetDistrictUrl(flatsPriceData.iloc[index][0], flatsPriceData.iloc[index][1])
        globalDataFrame = GetDistrictPropertyData(districtUrl, flatsPriceData.iloc[index][0], flatsPriceData.iloc[index][1], globalDataFrame)
        print(index)
    return globalDataFrame

In [309]:
def ChangeDataTypesAndCleanData(df):
    df = df.reset_index()
    df["Price"] = pd.to_numeric(df["Price"])
    df["PricePerM2"] = pd.to_numeric(df["PricePerM2"])
    df["Area"] = pd.to_numeric(df["Area"])
    df["floor"] = pd.to_numeric(df["floor"])
    df["Number of floors"] = pd.to_numeric(df["Number of floors"])
    df["Room count"] = pd.to_numeric(df["Room count"])
    df['Built year'] = df['Built year'].apply(lambda x: x[0:4])
    df["Built year"] = pd.to_datetime(df['Built year'].apply(lambda x: x[0:4]) + "-01-01")
    df["Equipment"] = df["Equipment"].apply(lambda x: "Partial decoration" if "USEFUL" in x else x)
    df["NearestKindergarten"] = pd.to_numeric(df["NearestKindergarten"])
    df["NearestEducational"] = pd.to_numeric(df["NearestEducational"])
    df["NearestShop"] = pd.to_numeric(df["NearestShop"])
    df["NearestStop"] = pd.to_numeric(df["NearestStop"])
    df["CrimeRate"] = pd.to_numeric(df["CrimeRate"])
    return df

In [311]:
globalDataFrame = pd.DataFrame()
globalDataFrame = GetOneSpecificDistrictPropertyData('https://en.aruodas.lt/butai-vilniuje-santariskese-dangerucio-g-parduodamas-6669-kv-m-triju-kambariu-butas-1-3243533/?return_url=%2Fbutai%2Fvilniuje%2F%3Fchange_region%3D1%26obj%3D1%26FRegion%3D461%26FDistrict%3D1', 'test1', 'test2', globalDataFrame)
globalDataFrame = ChangeDataTypesAndCleanData(globalDataFrame)
globalDataFrame

,index,Price,PricePerM2,Area,floor,Number of floors,Room count,Built year,Heating,Equipment,NearestKindergarten,NearestEducational,NearestShop,NearestStop,CrimeRate,City,District,href
0,0,167000,2504,66.69,3,4,3,2022-01-01,"Electric, aerothermal",Partial decoration,1.6,2.7,0.56,0.59,0,test1,test2,https://en.aruodas.lt/butai-vilniuje-santarisk...


In [288]:
globalDataFrame